# Optimised Engine Pipeline

The notebook provide an optimised version of the generic engine pipeline.

This notebook aims to enhance the engine pipeline through:

(1) Be compatible with Lambda Stack (https://lambdalabs.com/lambda-stack-deep-learning-software) to enable containisation, cloud and cross-platform development

(2) Upgrade tensorflow from 2.10 to 2.11. Version 2.13.0 is also compatible with Keras version 2.12.0.

(3) Enable execution on linux and Windows 11 WSL2 environments

(3) Minimise library dependencies

(4) Solve the parallel pipeline execution issues inherent in the generic_engine_pipeline for faster execution of the pipeline

Author: akudilczak

In [1]:
# pip install openl3

In [2]:
########################################################################################
# library imports
########################################################################################

# disable warnings
import warnings
warnings.filterwarnings("ignore")

# environment settings
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

# generic libraries
from platform import python_version
import functools
from functools import lru_cache
import diskcache as dc
import hashlib
import numpy as np
import datetime
import random
import io
import matplotlib.pyplot as plt

# tensor flow / keras related libraries
from tensorflow import keras
import keras
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa 
from keras.utils import dataset_utils

# image processing related libraries
import librosa

# audio processing libraries
import audiomentations
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

# print system information
print('Python Version           : ', python_version())
print('TensorFlow Version       : ', tf.__version__)
print('Keras Version            : ', keras.__version__)

print('Librosa Version          : ', librosa.__version__)
print('Audiomentations Version  : ', audiomentations.__version__)

Python Version           :  3.10.12
TensorFlow Version       :  2.13.0
Keras Version            :  2.13.1
Librosa Version          :  0.10.1
Audiomentations Version  :  0.33.0


## System Configuration

The following code sets up the pipeline with configuration options.

The key is to set the audio data directory to the root directory containing the folders with raw audio files. 

This expects the folders names to be the species names.  

In [3]:
########################################################################################
# system constants
########################################################################################

SC = {
    'AUDIO_DATA_DIRECTORY': "/Volumes/UGREEN Samsung SSD/project echo audio/test data/eBird Testing Subset Sorted sound",
    'CACHE_DIRETORY': "/Volumes/UGREEN Samsung SSD/project echo audio/test data/Temp Cache",

    'AUDIO_CLIP_DURATION': 2, # seconds
    'AUDIO_SAMPLE_RATE': 48000,

    'USE_DISK_CACHE': True,
    'SAMPLE_VARIANTS': 20,
    'CLASSIFIER_BATCH_SIZE': 16,
    'MAX_EPOCHS': 5000,
}


In [4]:
def enforce_memory_limit(mem_mb):
  # enforce memory limit on GPU

  gpus = tf.config.experimental.list_physical_devices('GPU')
  if gpus:
    try:
      tf.config.experimental.set_virtual_device_configuration(
          gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=mem_mb)])
      print(f"vram limit set to {mem_mb}MB")
    except RuntimeError as e:
      print(e)
      
# enforce max 5GB memory on GPU for this notebook if you have a small GPU
# enforce_memory_limit(5120)

# Disk Caching

The following code creates a disk cache.  You will need lots of space (20 GB+) if you create large melspectrograms.

The caching works by serialising a function call signature and hashing it into a key.  This key is used to store the result of the function call.

This allows the a result from the cache to be utilised instead of calling the function, which means the entire data processing pipeline can be cached if used correctly.

This works best when the function being cached is idempotent.  There may be circumstances where it doesn't matter.  Be careful with using this cache as you may get unexpected results.

In [5]:
########################################################################################
# Create a DiskCache instance
# This cache will allow us store intermediate function results to speed up the 
# data processing pipeline
########################################################################################
if SC['USE_DISK_CACHE']:
    cache = dc.Cache(SC['CACHE_DIRETORY'], cull_limit=0, size_limit=10**9) 


########################################################################################
# a helper function to create a hash key from a function signature and arguments
########################################################################################
def create_function_key(func, *args, **kwargs):
    partial_func = functools.partial(func, *args, **kwargs)
    func_name = partial_func.func.__name__
    func_module = partial_func.func.__module__
    args_repr = repr(partial_func.args)
    kwargs_repr = repr(sorted(partial_func.keywords.items()))

    key = f"{func_module}.{func_name}:{args_repr}:{kwargs_repr}"
    # Use hashlib to create a hash of the key for shorter and consistent length
    key_hash = hashlib.sha256(key.encode()).hexdigest()

    return key, key_hash, partial_func

# Loading the files into a Tensorflow dataset structure for model training

This initial code loads only the filenames.  The filenames are then split into train, validation and test datasets.  This is designed deliberately this way to conserve runtime memory.

Subsequent downstream loading of the file content occurs as part of the data pipeline transformation 'map' function.  See below.

In [6]:
########################################################################################
# these helper functions load the audio data into a 'dataset' using only paths
# just dealing with paths at this early stage means the entire dataset can be shuffled in
# memory and split before loading the actual audio data into memory
########################################################################################
def paths_and_labels_to_dataset(audio_paths, labels, num_classes):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    label_ds = dataset_utils.labels_to_dataset(
        labels, 
        'categorical', 
        num_classes)
    zipped_path_ds = tf.data.Dataset.zip((path_ds, label_ds))
    return zipped_path_ds

def create_datasets(audio_files, train_split=0.7, val_split=0.2):
    file_paths, labels, class_names = dataset_utils.index_directory(
            audio_files,
            labels="inferred",
            formats=('.ogg','.mp3','.wav','.flac'),
            class_names=None,
            shuffle=True,
            seed=42,
            follow_links=False)

    dataset = paths_and_labels_to_dataset(
        audio_paths=file_paths,
        labels=labels,
        num_classes=len(class_names))
    
    # Calculate the size of the dataset
    dataset_size = len(dataset)
    
    # Calculate the number of elements for each dataset split
    train_size = int(train_split * dataset_size)
    val_size = int(val_split * dataset_size)
    test_size = dataset_size - train_size - val_size

    # Split the dataset
    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size).take(val_size)
    test_ds = dataset.skip(train_size + val_size).take(test_size)
    
    return train_ds, val_ds, test_ds, class_names

In [7]:
# create the dataset
train_ds, val_ds, test_ds, class_names = create_datasets(SC['AUDIO_DATA_DIRECTORY'],train_split=0.8, val_split=0.19)
print("Class names: ", class_names)
print(f"Training   dataset length: {len(train_ds)}")
print(f"Validation dataset length: {len(val_ds)}")
print(f"Test       dataset length: {len(test_ds)}")

Found 2474 files belonging to 105 classes.
Class names:  ['Acanthagenys rufogularis', 'Acanthiza apicalis', 'Acanthiza lineata', 'Acanthiza pusilla', 'Acanthorhynchus tenuirostris', 'Accipiter novaehollandiae', 'Acridotheres tristis', 'Acrocephalus australis', 'Aegotheles cristatus', 'Alauda arvensis', 'Amaurornis moluccana', 'Anas castanea', 'Anthochaera chrysoptera', 'Aprosmictus erythropterus', 'Atrichornis clamosus', 'Bubulcus ibis', 'Cacomantis flabelliformis', 'Calamanthus fuliginosus', 'Caligavis chrysops', 'Caprimulgus macrurus', 'Charadrius ruficapillus', 'Chenonetta jubata', 'Colluricincla harmonica', 'Conopophila albogularis', 'Cormobates leucophaea', 'Corvus coronoides', 'Corvus orru', 'Cracticus nigrogularis', 'Cracticus torquatus', 'Dacelo novaeguineae', 'Dasyornis longirostris', 'Dicaeum hirundinaceum', 'Dicrurus bracteatus', 'Elanus axillaris', 'Entomyzon cyanotis albipennis', 'Eurostopodus mystacalis', 'Falco cenchroides', 'Falcunculus frontatus', 'Gallinula tenebrosa'

2023-09-16 15:24:18.054556: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2023-09-16 15:24:18.054578: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2023-09-16 15:24:18.054585: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2023-09-16 15:24:18.054612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-16 15:24:18.054627: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
# show what the pipeline looks like at this stage
for item in train_ds.take(10):
    print(item)

(<tf.Tensor: shape=(), dtype=string, numpy=b'/Volumes/UGREEN Samsung SSD/project echo audio/test data/eBird Testing Subset Sorted sound/Malurus splendens/region_137.200-139.200.wav'>, <tf.Tensor: shape=(105,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'/Volumes/UGREEN Samsung SSD/project echo audio/test data/eBird Testing Subset Sorted sound/Stomiopera unicolor/region_60.450-62.200.wav'>, <tf.Tensor: shape=(105,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

## Loading the audio files

In [9]:
from pydub import AudioSegment
# enable this cache if using large audio files, and lots of available RAM
def python_load_and_decode_file(sample, label, variant, cache_key, cache_found):
    if sample.decode('utf-8').endswith('.mp3'):
        audio = AudioSegment.from_mp3(sample.decode('utf-8'))
        sample = sample.decode('utf-8').replace('.mp3', '.wav')
        audio.export(sample, format="wav")

    if cache_found == np.int32(0):
        
        tmp_audio_t = None
        
        with open(sample, 'rb') as file:

            # Load the audio data with librosa
            tmp_audio_t, _ = librosa.load(file, sr=SC['AUDIO_SAMPLE_RATE'])
            
            # cast and keep right channel only
            if tmp_audio_t.ndim == 2 and tmp_audio_t.shape[0] == 2:
                tmp_audio_t = tmp_audio_t[1, :]
            
            # cast and keep right channel only
            tmp_audio_t = tmp_audio_t.astype(np.float32)
                    
            assert(tmp_audio_t is not None)
            assert(isinstance(tmp_audio_t, np.ndarray))
        
        sample = tmp_audio_t
        
    else:
        sample = cache[cache_key.decode('utf-8')]
    
    return sample, label, variant, cache_key, cache_found



# Sampling the audio files

The function loads and decodes an audio file from the given path, calculates the audio file's duration in seconds, and then extracts a random subsection of the specified duration (in seconds) from the audio. If the audio duration is shorter than the specified duration, the function pads the subsection with silence to meet the required length. The resulting subsection is returned as a tensor.

In [10]:
def tensorflow_load_random_subsection(sample, label, variant, cache_key, cache_found):
    
    if cache_found == np.int32(0):
        duration_secs = SC['AUDIO_CLIP_DURATION']
        
        # Determine the audio file's duration in seconds
        audio_duration_secs = tf.shape(sample)[0] / SC['AUDIO_SAMPLE_RATE']
        
        if audio_duration_secs>duration_secs:
        
            # Calculate the starting point of the 5-second subsection
            max_start = tf.cast(audio_duration_secs - duration_secs, tf.float32)
            start_time_secs = tf.random.uniform((), 0.0, max_start, dtype=tf.float32)
            
            start_index = tf.cast(start_time_secs * SC['AUDIO_SAMPLE_RATE'], dtype=tf.int32)
    
            # Load the 5-second subsection
            end_index = tf.cast(start_index + tf.cast(duration_secs, tf.int32) * SC['AUDIO_SAMPLE_RATE'], tf.int32)
            
            subsection = sample[start_index : end_index]
        
        else:
            # Pad the subsection with silence if it's shorter than 5 seconds
            padding_length = duration_secs * SC['AUDIO_SAMPLE_RATE'] - tf.shape(sample)[0]
            padding = tf.zeros([padding_length], dtype=sample.dtype)
            subsection = tf.concat([sample, padding], axis=0)

        sample = subsection

    return sample, label, variant, cache_key, cache_found

# Audio augmentations

The following code applies a sequence of augmentations to the audio signal.

A probability of applying the augmentation is used to ensure the augmentation isn't applied every sample.

This means there will be some samples that go straight through with no augmentations and a small probability that in fact all augmentations will be applied.

In [11]:
# Audio augmentation pipeline
def python_audio_augmentations(sample, label, variant, cache_key, cache_found):
    
    if cache_found == np.int32(0):
        # See https://github.com/iver56/audiomentations for more options
        augmentations = Compose([
            # Add Gaussian noise with a random amplitude to the audio
            # This can help the model generalize to real-world scenarios where noise is present
            AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.2),

            # Time-stretch the audio without changing its pitch
            # This can help the model become invariant to small changes in the speed of the audio
            TimeStretch(min_rate=0.8, max_rate=1.25, p=0.2),

            # Shift the pitch of the audio within a range of semitones
            # This can help the model generalize to variations in pitch that may occur in real-world scenarios
            PitchShift(min_semitones=-4, max_semitones=4, p=0.2),

            # Shift the audio in time by a random fraction
            # This can help the model become invariant to the position of important features in the audio
            Shift(min_fraction=-0.5, max_fraction=0.5, p=0.2),
        ])
        
        # apply audio augmentation to the clip
        # note: this augmentation is NOT applied in the test and validation pipelines
        sample = augmentations(samples=sample, sample_rate=SC['AUDIO_SAMPLE_RATE'])
    
    return sample, label, variant, cache_key, cache_found

# Image Augmentations

After the melspectrogram pipeline is executed, a 2D image is created of the signal energy at frequence/time points.

This image is augmented by applying the function below.  In this example a random rotation of between -2 and 2 degrees is applied every time.

In [12]:
# image augmentation pipeline
#def tensorflow_image_augmentations(sample, label, variant, cache_key, cache_found):

   # if cache_found == np.int32(0):
        # random rotation -2 deg to 2 deg
     #   degrees = tf.random.uniform(shape=(1,), minval=-2, maxval=2)
        
        # convert the angle in degree to radians
    #    radians = degrees * 0.017453292519943295  

        # rotate the image
     #   sample = tfa.image.rotate(sample, radians, interpolation='bilinear')
    
   # return sample, label, variant, cache_key, cache_found

In [13]:
def openl3_feature_extraction_pipeline(sample, label, variant, cache_key, cache_found):
    if cache_found == np.int32(0):
        # Adjust the length of the sample to match the expected input shape of the model
        sample = sample[:48000]  # Take the first 48000 samples
        sample = np.pad(sample, (0, 48000 - len(sample)), 'constant')  # Pad with zeros if less than 48000
        sample = sample.reshape(1, -1)  # Reshape to match the expected input shape
        
        # Call the OpenL3 model to extract the features from the audio data
        features = openl3_model.predict(sample[np.newaxis, ...])
        
        # OpenL3 model will return the features as a numpy array, 
        sample = features[0]
    
    return sample, label, variant, cache_key, cache_found



# Individual dataset pipelines

The dataset is split into different pipeline function as that augmentations would be applied to the training data only.

In [14]:
def tensorflow_add_variant_and_cache(path, label):
    variant     = tf.random.uniform(shape=(), minval=0, maxval=SC['SAMPLE_VARIANTS'], dtype=tf.int32)
    sample      = path
    cache_key   = b'no key'
    cache_found = np.int32(0)
    return sample, label, variant, cache_key, cache_found

def tensorflow_drop_variant_and_cache(sample, label, variant, cache_key, cache_found):
    return sample, label

def tensorflow_output_shape_setter(sample, label, variant, cache_key, cache_found):
    # Set the output shape based on the output shape of the OpenL3 model
    sample.set_shape([6144])  
    label.set_shape([len(class_names),]) 
    return sample, label, variant, cache_key, cache_found


In [15]:
def python_fuction_wrapper(pipeline_fn, out_types, sample, label, variant, cache_key, cache_found):

    # Use a lambda function to pass two arguments to the function
    sample, label, variant, cache_key, cache_found = tf.numpy_function(
        func=lambda v1,v2,v3,v4,v5: pipeline_fn(v1,v2,v3,v4,v5),
        inp=(sample, label, variant, cache_key, cache_found),
        Tout=out_types)

    return sample, label, variant, cache_key, cache_found

In [16]:
def python_disk_cache_start(sample, label, variant, cache_key, cache_found):

    cache_key   = b'no key'
    cache_found = np.int32(0)
    
    if SC['USE_DISK_CACHE']:
        _,cache_key,_ = create_function_key(python_disk_cache_start, sample, label, variant)
        if cache_key in cache:
            #print(f'found {cache_key} in cache')
            cache_found = np.int32(1)
        else:
            pass
            #print(f'{cache_key} not found in cache')
            
    return sample, label, variant, cache_key, cache_found

def python_disk_cache_end(sample, label, variant, cache_key, cache_found):
    cache_key = cache_key.decode('utf-8')
    if SC['USE_DISK_CACHE']:
        # if it was not found in the cache at the start, then populate with what we built
        # during the pipeline execution
        if cache_found == np.int32(0):
            #print(f'adding {cache_key} to cache')
            cache[cache_key] = sample
        #else:
        #    sample = cache[cache_key]
            
    return sample, label, variant, cache_key, cache_found

def python_openl3_feature_extraction(sample, label, variant, cache_key, cache_found):
    
    if cache_found == np.int32(0):
        # Adjust the length of the sample to match the expected input shape of the model
        sample = sample[:48000]  # Take the first 48000 samples
        sample = np.pad(sample, (0, 48000 - len(sample)), 'constant')  # Pad with zeros if less than 48000
        sample = sample.reshape(1, -1)  # Reshape to match the expected input shape
        
        # The OpenL3 model expects the input shape to be (batch, time, channel), 
        # so we need to expand the dimensions of the sample to match this.
        sample_expanded = np.expand_dims(sample, axis=0)
        
        # Use the OpenL3 model to extract the features
        features = openl3_model.predict(sample_expanded)
        
        # We then squeeze the output to remove the batch dimension
        sample = np.squeeze(features, axis=0)
    
    return sample, label, variant, cache_key, cache_found


import openl3
openl3_model = openl3.models.load_audio_embedding_model(input_repr="mel256", content_type="music", embedding_size=6144)

In [17]:
########################################################################################
# Create the datasets necessary for training a classification model
# Note: python and tensorflow functions are treated differently in the tensorflow
# pipeline.  Each python function needs to be wrapped. 
# this is why each pipeline function starts with python_ or tensorflow_ to make it clear 
########################################################################################

# Get the length of the training dataset
len_train_ds = len(train_ds)
parallel_calls = tf.data.AUTOTUNE
cache_output_types = (tf.string,tf.float32,tf.int32,tf.string,tf.int32)
procs_output_types = (tf.float32,tf.float32,tf.int32,tf.string,tf.int32)

# Create the training dataset pipeline
train_dataset = (train_ds
                 .shuffle(len_train_ds)
                 .map(tensorflow_add_variant_and_cache, num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, python_disk_cache_start, cache_output_types), num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, python_load_and_decode_file, procs_output_types), num_parallel_calls=parallel_calls)
                 .map(tensorflow_load_random_subsection, num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, python_audio_augmentations, procs_output_types), num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, openl3_feature_extraction_pipeline, procs_output_types), num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, python_openl3_feature_extraction, procs_output_types), num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, python_disk_cache_end, procs_output_types), num_parallel_calls=parallel_calls)
                 .map(tensorflow_output_shape_setter, num_parallel_calls=parallel_calls)
                 .map(tensorflow_drop_variant_and_cache, num_parallel_calls=parallel_calls)
                 .batch(SC['CLASSIFIER_BATCH_SIZE'])
                 .prefetch(parallel_calls)
                 .repeat(count=1)
)

In [18]:
# Create the validation dataset pipeline
validation_dataset = (val_ds
                    .map(tensorflow_add_variant_and_cache, num_parallel_calls=parallel_calls)
                    .map(functools.partial(python_fuction_wrapper, python_disk_cache_start, cache_output_types), num_parallel_calls=parallel_calls)
                    .map(functools.partial(python_fuction_wrapper, python_load_and_decode_file, procs_output_types), num_parallel_calls=parallel_calls)
                    .map(tensorflow_load_random_subsection, num_parallel_calls=parallel_calls)
                    .map(functools.partial(python_fuction_wrapper, openl3_feature_extraction_pipeline, procs_output_types), num_parallel_calls=parallel_calls)
                    .map(functools.partial(python_fuction_wrapper, python_openl3_feature_extraction, procs_output_types), num_parallel_calls=parallel_calls)
                    .map(tensorflow_output_shape_setter, num_parallel_calls=parallel_calls)
                    .map(functools.partial(python_fuction_wrapper, python_disk_cache_end, procs_output_types), num_parallel_calls=parallel_calls)
                    .map(tensorflow_output_shape_setter, num_parallel_calls=parallel_calls)
                    .map(tensorflow_drop_variant_and_cache, num_parallel_calls=parallel_calls)
                    .batch(SC['CLASSIFIER_BATCH_SIZE'])
                    .prefetch(parallel_calls)
                    .repeat(count=1)
)


In [19]:
# Create the test dataset pipeline
test_dataset = (test_ds
                 .map(tensorflow_add_variant_and_cache, num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, python_disk_cache_start, cache_output_types), num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, python_load_and_decode_file, procs_output_types), num_parallel_calls=parallel_calls)
                 .map(tensorflow_load_random_subsection, num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, openl3_feature_extraction_pipeline, procs_output_types), num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, python_openl3_feature_extraction, procs_output_types), num_parallel_calls=parallel_calls)
                 .map(tensorflow_output_shape_setter, num_parallel_calls=parallel_calls)
                 .map(functools.partial(python_fuction_wrapper, python_disk_cache_end, procs_output_types), num_parallel_calls=parallel_calls)
                 .map(tensorflow_output_shape_setter, num_parallel_calls=parallel_calls)
                 .map(tensorflow_drop_variant_and_cache, num_parallel_calls=parallel_calls)
                 .batch(SC['CLASSIFIER_BATCH_SIZE'])
                 .prefetch(parallel_calls)
                 .repeat(count=1)
)

# Data Pipeline Inspection

The following shows 1 batch of training data to provide a visual guide as to whether the data pipeline is functioning as expected.

It is important to observe in the ground truth labels that the classes are random - indicating that the pipeline shuffle function is working as expected.

In [20]:
# show what the pipeline looks like at this stage
for features, label in train_dataset.take(1):
    print(f'Sample info: {features.shape}, \nLabel info: {label.shape} \n{label}')
    for example in range(features.shape[0]):
        print(f'Example {example} features: {features[example]}')
        print(f'Example {example} label: {label[example]}')


2023-09-16 15:24:21.027573: W tensorflow/core/framework/op_kernel.cc:1816] INVALID_ARGUMENT: TypeError: Shift.__init__() got an unexpected keyword argument 'min_fraction'
Traceback (most recent call last):

  File "/Applications/anaconda3/envs/TF_projectecho_test/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/Applications/anaconda3/envs/TF_projectecho_test/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/var/folders/nr/hwp3lz852qn_89d92mtrsg640000gp/T/__autograph_generated_filei5btv_0o.py", line 10, in <lambda>
    (sample, label, variant, cache_key, cache_found) = ag__.converted_call(ag__.ld(tf).numpy_function, (), dict(func=ag__.autograph_artifact(lambda v1, v2, v3, v4, v5: ag__.converted_call(ag__.ld(pipeline_fn), (ag__.ld(v1), ag__.ld(v2), ag__.ld(v3), ag__.ld(v4), ag__.ld(v5)), None, fscope)), inp=(ag__.ld(sample), ag__.ld(

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: Shift.__init__() got an unexpected keyword argument 'min_fraction'
Traceback (most recent call last):

  File "/Applications/anaconda3/envs/TF_projectecho_test/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/Applications/anaconda3/envs/TF_projectecho_test/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/var/folders/nr/hwp3lz852qn_89d92mtrsg640000gp/T/__autograph_generated_filei5btv_0o.py", line 10, in <lambda>
    (sample, label, variant, cache_key, cache_found) = ag__.converted_call(ag__.ld(tf).numpy_function, (), dict(func=ag__.autograph_artifact(lambda v1, v2, v3, v4, v5: ag__.converted_call(ag__.ld(pipeline_fn), (ag__.ld(v1), ag__.ld(v2), ag__.ld(v3), ag__.ld(v4), ag__.ld(v5)), None,

2023-09-16 15:24:24.365926: W tensorflow/core/framework/op_kernel.cc:1816] INVALID_ARGUMENT: TypeError: Shift.__init__() got an unexpected keyword argument 'min_fraction'
Traceback (most recent call last):

  File "/Applications/anaconda3/envs/TF_projectecho_test/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/Applications/anaconda3/envs/TF_projectecho_test/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/var/folders/nr/hwp3lz852qn_89d92mtrsg640000gp/T/__autograph_generated_filei5btv_0o.py", line 10, in <lambda>
    (sample, label, variant, cache_key, cache_found) = ag__.converted_call(ag__.ld(tf).numpy_function, (), dict(func=ag__.autograph_artifact(lambda v1, v2, v3, v4, v5: ag__.converted_call(ag__.ld(pipeline_fn), (ag__.ld(v1), ag__.ld(v2), ag__.ld(v3), ag__.ld(v4), ag__.ld(v5)), None, fscope)), inp=(ag__.ld(sample), ag__.ld(

# Classification Model

In order to test whether the pipeline is working a CNN based image classification model is constructed below.  This model leverages pre-trained model weights for the EfficientNetV2 feature model which generates a vector representation of 1000 floats for each input image.  Note that on first run this model tensorflow hub library will check if model weights are available, and if not, will automatically download them to your computer.  This may take a few minutes the first time this is run.

The output from the EfficientNetV2 model is then fed into 2 fully connected layers to perform the classification function.

In [ ]:
import openl3

def build_model():
    # Input shape, change to match the shape of the feature vectors
    input_shape = (6144,)
    
    # Input layer
    input_layer = tf.keras.layers.Input(shape=input_shape)
    
    # Model layers
    x = tf.keras.layers.BatchNormalization()(input_layer)
    x = tf.keras.layers.Dense(len(class_names) * 8, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(len(class_names) * 4, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.50)(x)
    output_layer = tf.keras.layers.Dense(len(class_names), activation=None)(x)
    
    # Create model
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    
    # Compile model (you can adjust the optimizer and loss function as necessary)
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                  metrics=["accuracy"])
    
    # Display the model summary
    model.summary()

    return model



# Model Training

The following sets up callbacks to track model training and performs the model fit to train the model.

The callbacks will ensure the best model weights (as defined by lowest validation loss) will be written to disk during training.  This is important as the model training could take several hours (12+ hours) to complete and any errors may cause the training loop to exit - so having the checkpoints acts as a backup.  The model will stop training when it sees no further improvement to the validation loss, after which it will restore the best weights found.  This allows the training to discover when it is overfit and stop further training.  This is why the number of epics is 10000.  It is expected the model training will end significantly earlier than this.

In [ ]:
if not os.path.exists('models/'):
    os.mkdir('models/')
if not os.path.exists('models/1'):
    os.mkdir('models/1')    
    
# Allow all the weights to be trained
model = build_model()

# Tensorboard for visualisation of results
log_dir = "tensorboard_logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Reduce learning rate to avoid overshooting local minima
lr_reduce_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                         factor=0.75,
                                                         patience=8, 
                                                         verbose=1,
                                                         mode='min',
                                                         cooldown=0, 
                                                         min_lr=1e-7)

# End the training if no improvement for 16 epochs in a row, then restore best model weights
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                  min_delta=0,
                                                  patience=16,
                                                  verbose=0,
                                                  mode="min",
                                                  baseline=None,
                                                  restore_best_weights=True)

# Save the best model as it trains
mcp_save = tf.keras.callbacks.ModelCheckpoint('models/checkpoint_generic_model.hdf5', 
                                               save_best_only=True, 
                                               monitor='val_loss', 
                                               mode='min')

# Clear the cache (make sure cache is defined correctly)
cache.clear()

# Fit the model to the training set
# This may take a long time depending on your machine and data size
model.fit(train_dataset, 
          validation_data=validation_dataset,
          callbacks=[lr_reduce_plateau, early_stopping, tensorboard_callback, mcp_save],
          epochs=SC['MAX_EPOCHS']) 


# Model Predictions on test data

The following provides an example of how to use the trained model to perform inference (predictions).  It calculates the best class and the probability for that class.

In [ ]:
# Function to predict class and probability given a prediction
def predict_class(predictions):
    # Get the index of the class with the highest predicted probability
    predicted_index = int(tf.argmax(tf.squeeze(predictions)).numpy())
    # Get the class name using the predicted index
    predicted_class = class_names[predicted_index]
    # Calculate the predicted probability for the selected class
    predicted_probability = 100.0 * tf.nn.softmax(predictions)[predicted_index].numpy()
    # Round the probability to 2 decimal places
    predicted_probability = str(round(predicted_probability, 2))
    return predicted_class, predicted_probability

# Display class names and run prediction on test entries
print(f'Class names: {class_names}')
for features, labels in test_dataset:
    # Generate predictions for the given features
    predictions = model.predict(features, verbose=0)

    # Iterate over each item in the batch
    for batch_idx in range(predictions.shape[0]):
        # Get the index of the true class
        true_index = int(tf.argmax(tf.squeeze(labels[batch_idx])).numpy())
        # Get the true class name using the true index
        true_class = class_names[true_index]

        # Predict class and probability using the prediction function
        predicted_class, predicted_probability = predict_class(predictions[batch_idx])

        print(f'True class      : {true_class}')
        print(f'Predicted class : {predicted_class}')
        print(f'Predicted probability : {predicted_probability}')

In [ ]:
# Save the full model for use with tensorflow serving
model.save('models/echo_model/1/', overwrite=True)